<a href="https://colab.research.google.com/github/gmagannaDevelop/segnet/blob/mru_tests/example_notebooks/Model_abstraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MultiResUNet
## DCI-Net
#### Gustavo Magaña López

Creación de una clase que facilite el entrenamiento serializado de las arquitecturas encontradas en DCI-NET/segmed.

\########################################################

In [1]:
!ls

sample_data


In [0]:
#dir()

In [2]:
!apt install jq

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libjq1 libonig4
The following NEW packages will be installed:
  jq libjq1 libonig4
0 upgraded, 3 newly installed, 0 to remove and 7 not upgraded.
Need to get 276 kB of archives.
After this operation, 930 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libonig4 amd64 6.7.0-1 [119 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libjq1 amd64 1.5+dfsg-2 [111 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 jq amd64 1.5+dfsg-2 [45.6 kB]
Fetched 276 kB in 1s (414 kB/s)
Selecting previously unselected package libonig4:amd64.
(Reading database ... 145674 files and directories currently installed.)
Preparing to unpack .../libon

In [3]:
## If changes are made to the repo, uninstall and
## a fresh pip install are required, so it seems.
#!pip uninstall segnet
!pip install git+https://github.com/gmagannaDevelop/segnet.git@mru_tests
##!pip install git+https://github.com/gmagannaDevelop/segnet.git@log_hotfix

  Cloning https://github.com/gmagannaDevelop/segnet.git (to revision mru_tests) to /tmp/pip-req-build-j894jg96
  Running command git clone -q https://github.com/gmagannaDevelop/segnet.git /tmp/pip-req-build-j894jg96
  Running command git checkout -b mru_tests --track origin/mru_tests
  Switched to a new branch 'mru_tests'
  Branch 'mru_tests' set up to track remote branch 'mru_tests' from 'origin'.
     |████████████████████████████████| 215kB 4.2MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 307kB 50.7MB/s 
     |████████████████████████████████| 747kB 53.4MB/s 
  Created wheel for segnet: filename=segnet-0.4-cp36-none-any.whl size=28011 sha256=f8751ab96a2896dd40ac64321010f9010c7bb1dbce8c91cf6d8d316cabb574bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-41su1jr9/wheels/f1/c1/51/0ee27f445c3a618413714a84d4f5500547456389437

Imports

In [5]:
"""
!pip install gputil
import GPUtil
GPUtil.showUtilization(all=False, attrList=None, useOldCode=False)
deviceIDs = GPUtil.getAvailable(order = 'first', limit = 1, maxLoad = 0.5, maxMemory = 0.5, includeNan=False, excludeID=[], excludeUUID=[])
deviceIDs

!pip install gpuinfo
from gpuinfo import GPUInfo as gpu
gpu.check_empty()
"""

"\n!pip install gputil\nimport GPUtil\nGPUtil.showUtilization(all=False, attrList=None, useOldCode=False)\ndeviceIDs = GPUtil.getAvailable(order = 'first', limit = 1, maxLoad = 0.5, maxMemory = 0.5, includeNan=False, excludeID=[], excludeUUID=[])\ndeviceIDs\n\n!pip install gpuinfo\nfrom gpuinfo import GPUInfo as gpu\ngpu.check_empty()\n"

In [4]:
!pip install gputil
import GPUtil

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=eb79e3935854742ea9126502272f5560d5bdf7d54bca4a425c5ea58f84846061
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [6]:
GPUtil.showUtilization(all=True, attrList=None, useOldCode=False)
#gpus = GPUtil.getAvailable(order='first', limit=999, maxLoad=0.1)
#deviceIDs = GPUtil.getAvailable(order = 'first', limit = 10, maxLoad = 0.5, maxMemory = 0.5, includeNan=False, excludeID=[], excludeUUID=[])

| ID | Name                 | Serial        | UUID                                     || GPU temp. | GPU util. | Memory util. || Memory total | Memory used | Memory free || Display mode | Display active |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  0 | Tesla P100-PCIE-16GB | 0324716027585 | GPU-d8f725ef-edb5-fb79-dd7f-c07ed571d147 ||       47C |        0% |           0% ||      16280MB |         0MB |     16280MB || Enabled      | Disabled       |


In [16]:
type(GPUtil.getGPUs()[0])

GPUtil.GPUtil.GPU

In [0]:

ggpu_dict: Dict[str,Any] = {}
for gpu in GPUtil.getGPUs():
    ggpu_dict.update({
        "ID": gpu.id,
        "Name": gpu.name,
        "Serial": gpu.serial,
        "UUID": gpu.uuid,
        "GPU temp.": gpu.load*100,
        "GPU util %": gpu.memoryUtil*100,
        "Memory Total (MB)": gpu.memoryTotal,
        "Memory used (MB)": gpu.memoryUsed,
        "Memory Free (MB)": gpu.memoryFree,
        "Display Mode": gpu.display_mode,
        "Display Active": gpu.display_active
    })

In [19]:
for i in ggpu_dict.values():
  print(type(i))

<class 'int'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'str'>
<class 'str'>


In [8]:
### System-related
import platform,socket,re,uuid,json,psutil # for getSystemInfo method
import sys
import os
import datetime 
from typing import Dict, Optional, List, Tuple, Any, NoReturn, Callable, Union, Type
#import importlib.util
###############################################################

### Machine learning specific
#import segmentation_models as sm
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import keras.models
import tensorflow as tf
###############################################################

### In-Out
from skimage import io as skio
import json 
import pandas as pd
import h5py
import glob
###############################################################

### Visualization
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")
###############################################################

### Numerical
import numpy as np
###############################################################

### Repo-specific (segnet)
from segnet.train import train_segnet
from segnet.models import unet
from segnet.models import multiresunet as mru
from segnet.models import multiresunet2 as mru2
from segnet.models import multiresunet3 as mru3
from segnet.utils import timing
from segnet.metrics import metrics as mts
#from segnet.segnet.train.train_segnet import train_segnet
#import segnet.segnet as segnet

### Data-related
from google.colab import drive, files
drive.mount('/content/drive/')
###############################################################


### Not an import, but mandatory to be defined here :
root_dir   = "drive/My Drive/DCI-Net"
_log_file  = os.path.join(root_dir, "time_logs.jsonl")
_log_file_path = "drive/My Drive/DCI-Net/time_logs.jsonl"

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


Class definition

# TODO :
1. Set all default hyperparameters as class atributes to eliminate bloating of all methods.

2. Create the comment logging system.

In [0]:
# FINAL :
class Segmed(object):
  """
    Deep Learning Model Generator Object.
  """

  # List of needed modules :
  import platform,socket,re,uuid,json,psutil # for getSystemInfo method
  import os
  import matplotlib.pyplot as plt
  plt.style.use("ggplot")
  import tensorflow as tf
  from typing import Tuple, Optional, Union, Dict, Any, NoReturn
  # End of needed modules

  # Definition of static methods, utility functions logically
  # related to the class, but not semantically.
  @staticmethod
  def assert_isdir(path: str) -> str:
    """ Returns argument `path` if it is a directory, 
    raises an exception otherwise.   
    """
    if os.path.isdir(path):
      return path
    else:
      raise Exception(f"Path '{path}' is not a directory.")
  
  @staticmethod
  def ls(path: str) -> List[str]:
    """ Wrapper for os.listdir(path) """
    return os.listdir(path)

  @staticmethod
  def assert_isfile(path: str) -> str:
    """ Docstring """
    if os.path.isfile(path):
      return path
    else:
      raise Exception(f"File '{path}' is not a regular file.")    

  @staticmethod
  def json_cast(x: Any) -> Union[str,Any]:
    """ ~Verify (via good 'ol duck-typing) if an object is JSON-serialisable~
    
    Returns :
          x,  if parameter `x` is JSON-serialisable, 
    
      str(x), if json.dumps(x) throws an exception.
    """
    if timing.is_jsonable(x):
      return x
    else:
      return str(x)

  def _getSystemInfo() -> Dict[str,str]:
    """ Get SystemInfo as a dictionary.
    Original code snippet can be found at :
    https://stackoverflow.com/questions/3103178/how-to-get-the-system-info-with-python 
    """
    try:
      info={}
      info['platform']=platform.system()
      info['platform-release']=platform.release()
      info['platform-version']=platform.version()
      info['architecture']=platform.machine()
      info['hostname']=socket.gethostname()
      info['processor']=platform.processor()
      info['ram']=str(round(psutil.virtual_memory().total / (1024.0 **3)))+" GB"
      return info
    except Exception as e:
      logging.exception(e)

  def get_gpu() -> Union[Dict[str,Union[str,float,int]],List[Dict[str,Union[str,float,int]]]]:
    """ """
    ggpu_dict: Dict[str,Any] = {}
    gpus: List[GPUtil.GPUtil.GPU] = GPUtil.getGPUs()

    if len(gpus) > 1:
      gpu_list: List[Dict[str,Union[str,float,int]]] = []
      for gpu in gpus:
        gpu_list.append({
          "ID": gpu.id,
          "Name": gpu.name,
          "Serial": gpu.serial,
          "UUID": gpu.uuid,
          "GPU temp.": gpu.load*100,
          "GPU util %": gpu.memoryUtil*100,
          "Memory Total (MB)": gpu.memoryTotal,
          "Memory used (MB)": gpu.memoryUsed,
          "Memory Free (MB)": gpu.memoryFree,
          "Display Mode": gpu.display_mode,
          "Display Active": gpu.display_active
        })
      return gpu_list
    elif len(gpus) == 1:
      gpu = gpus[0]
      return {
          "ID": gpu.id,
          "Name": gpu.name,
          "Serial": gpu.serial,
          "UUID": gpu.uuid,
          "GPU temp.": gpu.load*100,
          "GPU util %": gpu.memoryUtil*100,
          "Memory Total (MB)": gpu.memoryTotal,
          "Memory used (MB)": gpu.memoryUsed,
          "Memory Free (MB)": gpu.memoryFree,
          "Display Mode": gpu.display_mode,
          "Display Active": gpu.display_active
      }
    else:
      return {}

      
  # End of static methods definition.

  # Class attributes which will be used if necessary parameters are 
  # not passed to different methods.
  # Decided to use dict literals instead of the constructor because of this :
  # 
  __data_gen_args: Dict[str,Any] = {
      "rescale": 1.0 / 255.0,
      "validation_split": 0.2,
      "dtype": tf.float32,
  }

  __hyper_params: Dict[str,Any] = {
      "batch_size": 8,
      "epochs": 10,
      "steps_per_epoch": 10
  }

  __model_checkpoint_kw: Dict[str,Any] = {
      "monitor": "val_jaccard_index", 
      "verbose": 1, 
      "save_best_only": True, 
      "mode": "max"
  }

  __compiling_kw: Dict[str,Any] = {
      "loss": tf.keras.losses.binary_crossentropy,
      "optimizer": tf.keras.optimizers.Adam(),
      "metrics": [
        mts.jaccard_index, mts.dice_coef, 
        mts.O_Rate, mts.U_Rate, mts.Err_rate
      ]
  }
  # End of class attributes.

  def __init__(
      self, 
      model: tf.keras.Model,
      name: str,
      base_dir: str,
      data_path: str,
      author: str,
      seed: int = 1
  ) -> None:
    # Set instance attributes :
    ## Identifiers :
    self._model: tf.keras.Model = model
    self.__datetime: datetime.datetime = datetime.datetime.utcnow()
    self._date: str = str(self.__datetime).split(".")[0]
    self._name: str = name
    self._author: str = author
    ## Directories :
    ### Model :
    self._base_dir: str = self.assert_isdir(base_dir)
    self._instance_dir: str = os.path.join(self._base_dir, self.name)
    ### Data :
    self._data_path: str = self.assert_isdir(data_path)
    self._img_path: str = os.path.join(self._data_path, 'imgs')
    self._msks_path: str = os.path.join(self._data_path, 'msks')
    
    self.__seed: int = seed

    ## Create model directory :
    try:
      os.mkdir(self._instance_dir)
    except:
      raise Exception(f"Could not create directory `{self._instance_dir}` at `{self._base_dir}`")

    try:
      def __custom_print_to_summary_file(x):
        with open(self.summary_file, 'a') as f:
          f.write(f"{x}\n")
      self._model.summary(print_fn=__custom_print_to_summary_file)
    except:
      raise Exception(f"Could not dump `model.summary()` output to {self.summary_file}")
    
    ## Create an instance decorator, specifying _log_file_path
    self.__logged: Callable = timing.time_log(self.log_file)
    
    ## Decorate __compile method
    self.__compile: Callable = self.__logged(self.__compile)

    ## Log initial info to comment file :
    _initial_message = f"\n{self._author}, running '{self._name}' model instantiated at {self._date}"
    _initial_message += f" on system with specs :\n {json.dumps(self.system_info)}\n\n"
    with open(self.comment_file, "w") as f:
      f.write(_initial_message)


  def __getitem__(self, key):
    try:
      return getattr(self, f"_{key}")
    except AttributeError:
      print(f"Attribute `{key}` is not yet defined for this object (or does not exist)")
      print(f"Defined attributes are: {self.keys}")


  @property
  def methods(self) -> List[str]:
    """ """
    return [func for func in dir(self) if callable(getattr(self, func))]


  @property
  def keys(self) -> List[Any]:
    """ Get the keys (as a list) that can be used to index the Segmed object """
    return sorted(list(map(lambda x: x[1:], self.__dict__.keys())))


  @property
  def items(self) -> Dict[str,Any]:
    """ Get the items `{ key: value }` pairs oh the class. """
    return { key: self[key] for key in self.keys }


  @property
  def values(self) -> List[Any]:
    """ Get the values, i.e. object properties. """ 
    return [ self[key] for key in self.keys ]


  @property
  def name(self) -> str:
    """ Name of the model, composed of: name-(author)-(date) """
    return f"{self._name}-({self._author})-({self._date})"


  @property
  def snapshot_file(self) -> str:
    """ Name (full path) of the model snapshot file (model weights, .h5 format) """
    return os.path.join(self._instance_dir, f"{self.name}-model.h5")


  @property
  def history_file(self) -> str:
    """ Name (full path) of the training history file (.csv format) """
    return os.path.join(self._instance_dir, f"{self.name}-history.csv")


  @property
  def comment_file(self) -> str:
    """ Name (full path) of the comment file (.txt format) """
    return os.path.join(self._instance_dir, f"{self.name}-comments.txt")


  @property
  def log_file(self) -> str:
    """ Name (full path) of the log file (.jsonl format) """
    return os.path.join(self._instance_dir, f"{self.name}-function_calls.jsonl")


  @property
  def summary_file(self) -> str:
    """ Name (full path) of the summary file, 
    yielded by calling self._model.summary() 
    (.txt format) 
    """
    return os.path.join(self._instance_dir, f"{self.name}-summary.txt")


  @property
  def model_file(self) -> str:
    """ Name (full path) of the description of the model (.json format) """
    return os.path.join(self._instance_dir, f"{self.name}-model.json")


  @property
  def images_path(self) -> str:
    """ Path where images are stored. 
    Following the Keras convention it should contain a
    directory named `images`.
    """
    return self._img_path


  @property
  def masks_path(self) -> str:
    """ Path where segmentation masks are stored. 
    Following the Keras convention it should contain a
    directory named `masks`.
    """
    return self._msks_path


  @property
  def images(self) -> List[str]:
    """ A list of all the images contained in `self.images_path`/images/ """
    return self.ls(os.path.join(self.images_path, 'images'))


  @property
  def masks(self) -> List[str]:
    """ A list of all the segmentation masks contained in `self.masks_path`/masks/ """
    return self.ls(os.path.join(self.masks_path, 'masks'))


  @property
  def system_info(self) -> Dict[str,str]:
    """ Wrapper for Segmed._getSystemInfo """
    return Segmed._getSystemInfo()


  def create_train_test_generators(
      self, 
      data_gen_args: Optional[Dict[str, Any]] = None,
      hyper_params: Optional[Dict[str, Any]] =  None
  ) -> NoReturn:
    """ Generate the train/test split. 
      data_gen_args defaults to the following Dict :
        {
          "rescale": 1.0 / 255.0,
          "validation_split": 0.2,
          "dtype": tf.float32,
        }
    """
    self._data_gen_args: Dict[str,Any] = data_gen_args or Segmed.__data_gen_args
    self._hyper_params:  Dict[str,Any] = hyper_params  or Segmed.__hyper_params

    # Crea the training generators with the defined transformations
    self.__image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**self._data_gen_args)
    self.__mask_datagen  = tf.keras.preprocessing.image.ImageDataGenerator(**self._data_gen_args)
    
    # Decoreate the ImageDataGenerator.flow from directory method for logging.
    _img_gen = self.__logged(self.__image_datagen.flow_from_directory)
    # Take images from directories
    self._image_generator_train = _img_gen(
        self._img_path,
        class_mode=None,
        color_mode="rgb",
        batch_size=self._hyper_params["batch_size"],
        seed=self.__seed,
        subset="training",
    )
    
    _msk_gen = self.__logged(self.__mask_datagen.flow_from_directory)
    self._mask_generator_train = _msk_gen(
        self._msks_path,
        class_mode=None,
        color_mode="grayscale",
        batch_size=self._hyper_params["batch_size"],
        seed=self.__seed,
        subset="training",
    )
    # Combine both generators
    # This needs to be a generator of generators, see the following
    # https://github.com/tensorflow/tensorflow/issues/32357
    self._train_generator = (
        pair for pair in zip(self._image_generator_train, self._mask_generator_train)
    )

    # And now the validation generators
    self._image_generator_val = _img_gen(
        self._img_path,
        class_mode=None,
        color_mode="rgb",
        batch_size=self._hyper_params["batch_size"],
        seed=self.__seed,
        subset="validation",
    )

    self._mask_generator_val = _msk_gen(
        self._msks_path,
        class_mode=None,
        color_mode="grayscale",
        batch_size=self._hyper_params["batch_size"],
        seed=self.__seed,
        subset="validation",
    )

    # Combine both generators, with same issue as before
    self._val_generator = (
        pair for pair in zip(self._image_generator_val, self._mask_generator_val)
    )


  def show_img_and_mask(self, n: int = 5) -> NoReturn:
    """ Show `n` images and segmentation maps, side by side
    to verify that they are batched together.
    """
    for i, pair in zip(range(n), self._train_generator):
      plt.figure(0)
      plt.subplot(1, 2, 1)
      plt.imshow(pair[0][0, ...])
      plt.subplot(1, 2, 2)
      plt.imshow(pair[1][0, ..., 0])
      plt.show()

    
  def create_custom_callback(
      self, 
      model_checkpoint_kw: Optional[Dict[str,Any]] = None
  ) -> NoReturn:
    """ Define the checkpoint callback.
      Parameters (keyword arguments) used to call 
      `tf.keras.callbacks.ModelCheckpoint()` can be 
      specified as a dictionary (parameter `model_checkpoint_kw`).

      Parameters:
        model_checkpoint_kw : Optional, defaults to:
          Dict( 
            monitor=monitor, 
            verbose=1, 
            save_best_only=True, 
            mode="max"
          )
    """
    # Define the checkpoint callback, always maximum mode for custom metrics
    self._model_checkpoint_kw: Dict[str,Any] = model_checkpoint_kw or Segmed.__model_checkpoint_kw
      
    self._checkpoint = tf.keras.callbacks.ModelCheckpoint(
        self.snapshot_file, 
        **self._model_checkpoint_kw
    )


  def compile(
      self,
      compiling_kw: Optional[Dict[str,Any]] = None
  ) -> NoReturn:
    """ This eventually will be a wrapper """ 

    self._compiling_kw: Dict[str,Any] = compiling_kw or Segmed.__compiling_kw 
    self.__compile(**self._compiling_kw)


  def __compile(self, **compiling_kw):
    """ Compile the model with custom params, 
    creating a log 
    """
    self._model.compile(**compiling_kw)


  def train(
      self, 
      compiling_kw: Dict[str,Any] = None,
      model_checkpoint_kw: Optional[Dict[str,Any]] = None,
      data_gen_args: Optional[Dict[str, Any]] = None,
      hyper_params: Optional[Dict[str, Any]] =  None
  ) -> NoReturn:
    """ Docstring """

    self._compiling_kw:  Dict[str,Any] = compiling_kw  or Segmed.__compiling_kw
    self._data_gen_args: Dict[str,Any] = data_gen_args or Segmed.__data_gen_args
    self._hyper_params:  Dict[str,Any] = hyper_params  or Segmed.__hyper_params 
    self._model_checkpoint_kw: Dict[str,Any] = model_checkpoint_kw or Segmed.__model_checkpoint_kw

    self.compile(compiling_kw=compiling_kw)
    self.create_custom_callback(model_checkpoint_kw=model_checkpoint_kw)
    self.create_train_test_generators(data_gen_args=data_gen_args, hyper_params=hyper_params)
    
    # Decorate the model's fit generator to log parameters and execution time :
    _fit_generator = self.__logged(self._model.fit_generator)

    # Create history 
    self._history = _fit_generator(
        self._train_generator,
        callbacks=[self._checkpoint],
        verbose=1,
        validation_data=self._val_generator,
        validation_steps=self._hyper_params["steps_per_epoch"],
        steps_per_epoch=self._hyper_params["steps_per_epoch"],
        epochs=self._hyper_params["epochs"],
        use_multiprocessing=True
    )

    try:
      self._metrics_history = pd.DataFrame(self._history.history)
      self._metrics_history.to_csv(self.history_file)
      print(f"History saved to {self.history_file}")
    except:
      print(f"Could not open file : {self.history_file}")


  def comment(self, cmt: str) -> NoReturn:
    """ Comment something to self.comment_file, logging the Author and UTCdatetime """
    _now = str(datetime.datetime.utcnow()).split(".")[0]
    _cmt = f"\n{self._author} @ {_now} : \n\t{cmt}"
    with open(self.comment_file, 'a') as f:
      f.write(f"{_cmt} \n")


In [0]:
dataset_paths = {
    "isbi": "drive/My Drive/DCI-Net/Colab_data/ISBI_neural/structured", # Images as tiff volumes
    "colonoscopy": "drive/My Drive/DCI-Net/Colab_data/colonoscopy",     # Full original
    "dermoscopy80": "drive/My Drive/DCI-Net/Colab_data/dermoscopy80",   # reduced to 80 images
    "dermoscopy150": "drive/My Drive/DCI-Net/Colab_data/dermoscopy150", # reduced to 150, randomly 
                                                                          # sampled images.
    "chinese1": "drive/My Drive/DCI-Net/Colab_data/Dataset 2"           # Chinese dataset, Abdiel's
}

In [0]:
optimizers = {
    "chinese": tf.keras.optimizers.SGD(learning_rate=0.06, momentum=0.2, nesterov=False),
    "Original Adam": tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999, epsilon=10e-8)
}

In [23]:
optimizers["Original Adam"].get_config()

{'amsgrad': False,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'decay': 0.0,
 'epsilon': 1e-07,
 'learning_rate': 0.001,
 'name': 'Adam'}

In [0]:
my_compiling_kw = {
    'optimizer': optimizers["Original Adam"],
    'loss': 'binary_crossentropy',
    'metrics': [
      mts.jaccard_index, mts.dice_coef, 
      mts.O_Rate, mts.U_Rate, mts.Err_rate 
    ]
}

In [0]:
my_hyper_params = {
    'batch_size': 25,
    'epochs': 150,
    'steps_per_epoch': 6
}

In [26]:
models = {
    "Unet": unet(),
    "MultiResUNet Edwin": mru.MultiResUnet(),
    "MultiResUNet Gustavo": mru2.MultiResUNet(),
    "MultiResUNet Original": mru3.MultiResUnet()
}

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [27]:
models["MultiResUNet Gustavo"]

In [29]:
models.keys()

dict_keys(['Unet', 'MultiResUNet Edwin', 'MultiResUNet Gustavo', 'MultiResUNet Original'])

In [30]:
_model_name = list(models.keys())[2]
_model_name

'MultiResUNet Gustavo'

In [31]:
Segmed.get_gpu()

{'Display Active': 'Disabled',
 'Display Mode': 'Enabled',
 'GPU temp.': 0.0,
 'GPU util %': 0.0,
 'ID': 0,
 'Memory Free (MB)': 16280.0,
 'Memory Total (MB)': 16280.0,
 'Memory used (MB)': 0.0,
 'Name': 'Tesla P100-PCIE-16GB',
 'Serial': '0324716027585',
 'UUID': 'GPU-d8f725ef-edb5-fb79-dd7f-c07ed571d147'}

In [0]:
x = Segmed(
    model=models[_model_name],  # Instantiated from segmed.models
    name=_model_name,           # Self-explanatory
    base_dir=os.path.join(root_dir, "SegMedLogs"), # Where to create the directory
                                                   # that will contain the logs.
    data_path=dataset_paths["chinese1"], # See dataset_paths to analise the dataset.
    author='Gustavo Magaña'
) 

In [0]:
x.comment(" ")

In [0]:
x.comment(" ")

In [0]:
x.train(compiling_kw=my_compiling_kw, hyper_params=my_hyper_params)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Found 80 images belonging to 1 classes.
Found 80 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Epoch 1/1000


ResourceExhaustedError: ignored

In [0]:
x.create_train_test_generators()

Found 64 images belonging to 1 classes.
Found 64 images belonging to 1 classes.
Found 16 images belonging to 1 classes.
Found 16 images belonging to 1 classes.


In [0]:
from types import FunctionType

In [0]:
x.__dict__.items()

dict_items([('_model', <tensorflow.python.keras.engine.training.Model object at 0x7fec25f71c88>), ('_Segmed__datetime', datetime.datetime(2020, 1, 29, 6, 57, 19, 720278)), ('_date', '2020-01-29 06:57:19'), ('_name', 'multiresunet'), ('_snapshot_dir', 'drive/My Drive/DCI-Net/ModelSnapshots'), ('_history_dir', 'drive/My Drive/DCI-Net/Historiae'), ('_comment_dir', 'drive/My Drive/DCI-Net/Comments'), ('_data_path', 'drive/My Drive/DCI-Net/Colab_data/dermoscopy80'), ('_log_file_path', 'drive/My Drive/DCI-Net/time_logs.jsonl'), ('_img_path', 'drive/My Drive/DCI-Net/Colab_data/dermoscopy80/imgs'), ('_msks_path', 'drive/My Drive/DCI-Net/Colab_data/dermoscopy80/msks'), ('_Segmed__seed', 1), ('_author', 'Gustavo Magaña'), ('_Segmed__logged', <function time_log.<locals>.timed at 0x7fec2b8e9b70>), ('_Segmed__compile', <function Segmed.__compile at 0x7fec2b8e91e0>)])

In [0]:
[f for f, y in x.__dict__.items() if isinstance(y, FunctionType)]

['_Segmed__logged', '_Segmed__compile']

In [0]:
x.train(hyper_params=hyper_params)

Found 64 images belonging to 1 classes.
Found 64 images belonging to 1 classes.
Found 16 images belonging to 1 classes.
Found 16 images belonging to 1 classes.
Epoch 1/80


In [0]:
x.create_train_test_generators()

Found 489 images belonging to 1 classes.
Found 489 images belonging to 1 classes.
Found 122 images belonging to 1 classes.
Found 122 images belonging to 1 classes.


In [0]:
x.comment("fresh start")

In [0]:
!ls -t "drive/My Drive/DCI-Net/Comments/"

'multiresunet-(Gustavo Magaña)-(2020-01-29 07:18:39).txt'
'multiresunet-(Gustavo Magaña)-(2020-01-29 07:13:23).txt'
'multiresunet-(Gustavo Magaña)-(2020-01-29 06:38:59).txt'
'multiresunet-(Gustavo Magaña)-(2020-01-29 06:18:12).txt'
'multiresunet-(Gustavo Magaña)-(2020-01-29 06:16:09).txt'
'multiresunet-(Gustavo Magaña)-(2020-01-29 06:12:09).txt'
'multiresunet-(Gustavo Magaña)-(2020-01-29 04:11:56).txt'
'multiresunet-(Gustavo Magaña)-(2020-01-29 04:11:09).txt'


In [0]:
!cat "drive/My Drive/DCI-Net/Comments/multiresunet-(Gustavo Magaña)-(2020-01-29 07:18:39).txt"


Gustavo Magaña, running 'multiresunet' model instantiated at 2020-01-29 07:18:39on system with specs :
 {'platform': 'Linux', 'platform-release': '4.14.137+', 'platform-version': '#1 SMP Thu Aug 8 02:47:02 PDT 2019', 'architecture': 'x86_64', 'hostname': '2b5d4a3808d4', 'processor': 'x86_64', 'ram': '13 GB'}



In [0]:
x.train()

Found 80 images belonging to 1 classes.
Found 80 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Epoch 1/10
10/10 [==============================] - 4s 387ms/step - loss: 0.8121 - jaccard_index: 0.1390 - dice_coef: 0.2438 - O_Rate: 62914376.0000 - U_Rate: 0.0000e+00 - Err_rate: 62914376.0000

Epoch 00001: val_jaccard_index improved from -inf to 0.13901, saving model to drive/My Drive/DCI-Net/ModelSnapshots/multiresunet-(Gustavo Magaña)-(2020-01-28 19:45:06).h5
10/10 [==============================] - 84s 8s/step - loss: 0.5613 - jaccard_index: 0.2313 - dice_coef: 0.3736 - O_Rate: 0.0051 - U_Rate: 0.0666 - Err_rate: 0.2628 - val_loss: 0.8121 - val_jaccard_index: 0.1390 - val_dice_coef: 0.2438 - val_O_Rate: 62914376.0000 - val_U_Rate: 0.0000e+00 - val_Err_rate: 62914376.0000
Epoch 2/10
10/10 [==============================] - 1s 130ms/step - loss: 0.6509 - jaccard_index: 0.1387 - dice_coef: 0.2434 - O_Rate: 62914376.0000 - U_

In [0]:
dir(x["history"])
x["history"].params

{'batch_size': None,
 'do_validation': True,
 'epochs': 10,
 'metrics': ['loss',
  'jaccard_index',
  'dice_coef',
  'O_Rate',
  'U_Rate',
  'Err_rate',
  'val_loss',
  'val_jaccard_index',
  'val_dice_coef',
  'val_O_Rate',
  'val_U_Rate',
  'val_Err_rate'],
 'samples': 10,
 'steps': 10,
 'verbose': 1}

In [0]:
# Declare the paths to use (following the Keras convention)
# https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator

_model_log_path = os.path.join(
    _model_snapshots_dir, 
    yield_model_name(architecture="MultiResUNet")
)


Get the stored snapshots :